## Implicit



Now that our destructors get called automatically on scope exit, we want to
make sure that we don't need to do anything to make it happen.

After all, our current example has 3 lines of context-management to 2 lines of
actually functionality. It is also all too easy to forget to push your instance
into the `dtor_stack` and have them hang indefinitely.

If we could, we'd like our function to look like this:

In [ ]:
def main():
    greeter1 = Greeter(1)
    greeter2 = Greeter(2)

And have it implicitly do all the plumbing we managed earlier.

First, since we always want to push our objects onto the dtor stack, let's
make it part of their construction.

In [ ]:
class Greeter:
    def __init__(self, name, dtor_stack):
        self.name = name
        print(f"Hello, {self.name}!")

        dtor_stack.push(self)
    ...

class Greeter:
    def __init__(self, name, dtor_stack):
        dtor_stack.push(self)

        self.name = name
        print(f"Hello, {self.name}!")


    def __enter__(self):
        return self

    def __exit__(self, exc_type, exc_val, exc_tb):
        print(f"Goodbye, {self.name}.")
        return False


def main():
    with DtorScope() as dtor_stack:
        greeter1 = Greeter(1, dtor_stack)
        greeter2 = Greeter(2, dtor_stack)


main()

That's a good start, as we can no longer forget to do that.

That said, we're explicitly repeating and passing around a construct that
should be implicit. Let's fix that.

We wamt to convert our code from this:

In [ ]:
def main():
    with DtorScope() as dtor_stack:
        greeter1 = Greeter(1, dtor_stack)
        greeter2 = Greeter(2, dtor_stack)

To this:

In [ ]:
def main():
    with DtorScope():
        greeter1 = Greeter(1)
        greeter2 = Greeter(2)

To do that, we'll have to put the `DtorScope` somewhere our `Greeter`
class can find it.
A global variable!

And just like function calls go into a stack so that we know where to return,
so will our `DtorScope`s. So instead of a single global variable,
we'll have to use a global stack.

In [ ]:
_dtor_stack = []


def get_dtor_stack():
    return _dtor_stack


class DtorScope:
    def __init__(self):
        self.stack = []
        get_dtor_stack().append(self)

    def __enter__(self):
        return self

    def __exit__(self, exc_type, exc_val, exc_tb):
        get_dtor_stack().pop()

        while self.stack:
            self.stack.pop().__exit__(exc_type, exc_val, exc_tb)

    def push(self, cm):
        self.stack.append(cm)


def push_dtor(cm):
    return get_dtor_stack()[-1].push(cm)

This is the same as our previous dtor-scope, but now we keep a global
stack of scopes. This allows us to always tell which dtor-stack to push
our instances into without naming the stack.

In [ ]:

class Greeter:
    def __init__(self, name):
        self.name = name
        print(f"Hello, {self.name}!")

        push_dtor(self)

    def __enter__(self):
        return self

    def __exit__(self, exc_type, exc_val, exc_tb):
        print(f"Goodbye, {self.name}.")
        return False


def main():
    with DtorScope():
        greeter1 = Greeter(1)
        greeter2 = Greeter(2)

main()

Then a little tweak to our `Greeter`, and we're done.

Much better, but we still need to explicitly create the scope inside every
function.

Now, since the dtor scoping mechanism has nothing to do with the function itself,
we can use it at the callsite instead of inside the function,
and it'd work exactly the same.

In [ ]:
def main():
    greeter1 = Greeter(1)
    greeter2 = Greeter(2)

with DtorScope():
    main()

This will have to be done on _every_ callsite, so we add a utility function
to help with that.

The `*args, **kwargs` syntax is Python's equivalent of perfect-forwarding.

In [ ]:
def call(f, *args, **kwargs):
    with DtorScope():
        return f(*args, **kwargs)

call(main)

Alternatively, we can take the function and return a closure that includes
the scoping

In [ ]:
def cpp_function(f):
    def _wrapper(*args, **kwargs):
        with DtorScope():
            return f(*args, **kwargs)
    return _wrapper

scoped_main = cpp_function(main)

scoped_main()

Inside `_wrapper` we capture `f` from the parent scope.
As Python is reference-based, we don't need to specify how to capture.

Next, since Python is dynamic, we can replace the original `main`
function with the scoped one

In [ ]:
def cpp_function(f):
    def _wrapper(*args, **kwargs):
        with DtorScope():
            return call(f, *args, **kwargs)
    return _wrapper

def main():
    greeter1 = Greeter(1)
    greeter2 = Greeter(2)

main = cpp_function(main)

main()

Here the wrapper holds the previous version of the function, while the name
`main` is bound to the wrapped version.

In Python, wrapping a function and replacing its original is a common operation.
Because of that, we have some syntactic sugar called "decorators"

In [ ]:
def cpp_function(f):
    def _wrapper(*args, **kwargs):
        with DtorScope():
            return f(*args, **kwargs)

    return _wrapper

@cpp_function
def main():
    greeter1 = Greeter(1)
    greeter2 = Greeter(2)

main()

This is completely identical to the previous code.
The decorator syntax, however, makes things look declerative and natural.


This is much better. Now the insides of our function look just like a regular
function.